In [1]:
import random
import numpy as np
import pandas as pd

# ====== PARAMETERS (replace with data-sourced values) ======
SEASON_GAMES = 16
SIM_REPS = 100
NUM_TEAMS = 1

# Concussions per team per season (Poisson lambda)
lambda_team = 0.6  # placeholder, replace with sourced value

concussion_prob_by_pos = { # PLACEHOLDER
    'QB': 0.02,   # 2% chance per game for the QB on this team
    'RB': 0.03,
    'WR': 0.03,
    'TE': 0.015,
    'K': 0.001
}

# Position buckets and data-based probabilities
positions = ['QB', 'RB', 'WR', 'TE', 'K'] 

pos_probs = [
    7/55,   # QB
    10/55,  # RB
    20/55,  # WR
    18/55,  # TE
    0/55    # K
]

# Average games missed by position (from your research)
games_missed_by_position = {
    'QB': 2,
    'RB': 1,
    'WR': 1,
    'TE': 2,
    'K': 0
}

# Probability that the position "produces" their points_loss in a game (when available)
# Example: QB contributes their average 4.0 points in ~95% of games; adjust as needed.
# PLACEHOLDER
prob_score_by_pos = {
    'QB': 0.95,
    'RB': 0.60,
    'WR': 0.55,
    'TE': 0.30,
    'K': 0.50
}
# expected points lost per game by position (placeholder)
points_loss = {'QB':4.0, 'RB':1.5, 'WR':1.2, 'TE':0.5,'K':0.1}

# per-player per-game salary (median values) (NOT PLACEHOLDERS)
per_game_salary = {'QB': 600000, 'RB':158823, 'WR':142000, 'TE':121875, 'K':125000}

# points to wins conversion (NOT PLACEHOLDER)
POINTS_PER_WIN = 47.0

# optional: value per win (for revenue/valuation impact) (NOT PLACEHOLDER)
VALUE_PER_WIN = 2000000

# percent of fans not showing up when players are injured per position (placeholder)
FAN_DROP_PERCENT = {'QB':0.05, 'RB':0.03, 'WR':0.02, 'TE':0.01,'K':0.005}


In [5]:
# Random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# ========= Simulation ========
def simulate_season():
    total_points_lost = 0.0
    total_salary_lost = 0.0
    total_fan_drop = 0.0
    
    # Simulate number of concussions for the team this season
    num_concussions = np.random.poisson(lambda_team)
    
    for _ in range(num_concussions):
        # Determine position of injured player
        injured_pos = np.random.choice(positions, p=pos_probs)
        
        # Determine games missed
        games_missed = games_missed_by_position[injured_pos]
        
        # Calculate points lost
        points_lost = points_loss[injured_pos] * games_missed
        total_points_lost += points_lost
        
        # Calculate salary lost
        salary_lost = per_game_salary[injured_pos] * games_missed
        total_salary_lost += salary_lost
        
        # Calculate fan drop
        fan_drop = FAN_DROP_PERCENT[injured_pos] * games_missed
        total_fan_drop += fan_drop
    
    return total_points_lost, total_salary_lost, total_fan_drop

# Run simulations
results = []
for sim in range(SIM_REPS):
    points_lost, salary_lost, fan_drop = simulate_season()
    results.append({
        'Simulation': sim + 1,
        'Points_Lost': points_lost,
        'Salary_Lost': salary_lost,
        'Fan_Drop': fan_drop
    })
# print results
print(results)

[{'Simulation': 1, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 2, 'Points_Lost': 3.0, 'Salary_Lost': 317646.0, 'Fan_Drop': 0.06}, {'Simulation': 3, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 4, 'Points_Lost': 1.0, 'Salary_Lost': 243750.0, 'Fan_Drop': 0.02}, {'Simulation': 5, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 6, 'Points_Lost': 3.0, 'Salary_Lost': 317646.0, 'Fan_Drop': 0.06}, {'Simulation': 7, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 8, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 9, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 10, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 11, 'Points_Lost': 1.5, 'Salary_Lost': 158823.0, 'Fan_Drop': 0.03}, {'Simulation': 12, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, 'Fan_Drop': 0.0}, {'Simulation': 13, 'Points_Lost': 0.0, 'Salary_Lost': 0.0, '

have (throughout 3 years)
- head injuries of contributing players 
- amount of points scored while out
- amount of points in first game back
- the amount of games won and lost while gone 
- whether they won or lost first game back

want 
- chance of conncussion per position
- probability of each position making a point
- value per win 
- money lost while players are out